In [1]:
%pip install m3u8
%pip install pycryptodome
%pip install python-slugify==8.0.1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import m3u8
import requests
from Crypto.Util.Padding import pad
from Crypto.Cipher import AES
import requests
from tqdm import tqdm
import binascii
from http.cookies import SimpleCookie
from bs4 import BeautifulSoup
import re
from slugify import slugify

In [17]:

def get_input():
    """
    get url, video file name from user
    :return: raw_url(ends with ".mp4" or ".ts"), file_name(ends with ".mp4")
    """
    raw_url = input("enter the url you copied from the browser (ex. https://.../segment-1-v1-a1.ts) : ")
    file_name = input("enter the name of the video file will be downloaded (ex. video.mp4) : ")
    if file_name[-4:] != ".mp4":
        file_name += ".mp4"
    return raw_url, file_name

def get_base_url(raw_url):
    """
    truncate .ts segment
    :param raw_url: input url from user
    :return: base_url(ends with "mp4")
    """
    if ".ts" in raw_url:
        raw_url = "/".join(raw_url.split("/")[:-1])
    print(raw_url)
    return raw_url

def get_playlist(base_url):
    """
    fetch a playlist from base_url
    :param base_url: truncated url
    :return: m3u8 playlist object
    """
    tmp_playlist = m3u8.load(uri=base_url+"/index.m3u8")
    return tmp_playlist

def decrypt_video(encrypted_data, key, iv):
    """
    as a .ts file is encrypted with AES-128 (CBC) algorithm, decrypt it using pycryptodome
    :param encrypted_data: .ts file data
    :param key: fetched .key file
    :param iv: sequence number (iv is not specified)
    :return: decrypted segment
    """
    encrypted_data = pad(data_to_pad=encrypted_data, block_size=AES.block_size)
    aes = AES.new(key=key, mode=AES.MODE_CBC, iv=iv)
    decrypted_data = aes.decrypt(encrypted_data)
    return decrypted_data

def binify(x):
    """
    convert sequence number(int) to 128bit representation for AES-128 decryption
    """
    h = hex(x)[2:].rstrip('L')
    return binascii.unhexlify('0'*(32-len(h))+h)


def get_soup(lecture_url, cookie_str):
    cookie = SimpleCookie()
    cookie.load(cookie_str)
    cookies = {k: v.value for k, v in cookie.items()}
    resp = requests.get(lecture_url, cookies=cookies)
    soup = BeautifulSoup(resp.text, 'html.parser') if resp is not None else None
    return soup

def download_video(uri, file_name):
    playlist = m3u8.load(uri=uri)
    key = requests.get(playlist.keys[-1].absolute_uri).content
    seq_len = len(playlist.segments)
    for i in tqdm(range(seq_len)):
        seg = playlist.segments[i]
        data = requests.get(seg.absolute_uri).content
        iv = binify(i+1)
        data = decrypt_video(data, key, iv)
        with open(file_name,"ab" if i != 0 else "wb") as f:
            f.write(data)

def get_vod_list(lecture_soup, cookie_str):
    vod_list = []
    for li in tqdm(lecture_soup.find_all("li", {"class": "vod"})):
        a = li.find('a')
        video_name = a.find("span", {"class":"instancename"}).text
        viewer_url = re.search("window.open\(\'(.+?)\',", a['onclick']).group(1)
        vod_soup = get_soup(viewer_url, cookie_str)
        if vod_soup is not None:
            source = vod_soup.find("source")
            if source is not None:
                video_url = source["src"]
                vod_list.append({'name': video_name, 'url': video_url})
    return vod_list


In [29]:
cookie_str = input ("Enter cookie string:") 
lecture_url = input("Enter Lecture URL : ") 
lecture_soup = get_soup(lecture_url, cookie_str)
if lecture_soup is not None:
    vod_list = get_vod_list(lecture_soup, cookie_str)


In [30]:
# cookie_str = '_gcl_au=1.1.1450246283.1677722439; au_id=254dbcf42af70e0817e5981417f2bc828e5-613c; _gid=GA1.2.597159178.1678320723; ubboard_read=O%CF%0B%00e%12%93%81%A6%DF%13%A2P%0D; hide_menu=/mod/quiz,/mod/assign,/mod/vod,/mod/econtents; _gat_UA-160620709-35=1; MoodleSession=c61t326i6kb4i7c0gk3a5pb40v; _ga_M9Z4NCPPY6=GS1.1.1678403564.277.1.1678403568.0.0.0; login_check=true; _ga=GA1.2.358086843.1645707131'
# lecture_url = "https://ys.learnus.org/course/view.php?id=229677"



In [27]:
vod_list = []
for li in tqdm(soup.find_all("li", {"class": "vod"})):
    a = li.find('a')
    video_name = a.find("span", {"class":"instancename"}).text
    viewer_url = re.search("window.open\(\'(.+?)\',", a['onclick']).group(1)
    vod_soup = get_soup(viewer_url, cookie_str)
    if vod_soup is not None:
        source = vod_soup.find("source")
        if source is not None:
            video_url = source["src"]
            vod_list.append({'name': video_name, 'url': video_url})


100%|██████████| 29/29 [00:04<00:00,  6.16it/s]


In [28]:
import ipywidgets as widgets
from IPython.display import display
layout = widgets.Layout(width='auto', height='auto') #set width and height
output = widgets.Output()
list_checks = []
for item in vod_list:
    url = item['url']
    name = item['name']
    checkbox = widgets.Checkbox(value=True, description=name, indent=False, display='flex', layout=layout)
    list_checks.append(checkbox)
    display(checkbox, output)

button = widgets.Button(description="Download")
def on_button_clicked(b):
  # Display the message within the output widget.
    for i, check in enumerate(list_checks):
        url, name, checked = vod_list[i]['url'], vod_list[i]['name'], check.value
        name = slugify(name, allow_unicode=True, separator='_') + '.mp4'
        if checked:
            download_video(url, name)
button.on_click(on_button_clicked)
display(button, output)

Checkbox(value=True, description='[CSI4116] Intro to Computer Vision - Lecture 1 동영상', indent=False, layout=La…

Output()

Checkbox(value=True, description='[CSI4116] Intro to Computer Vision - Lecture 2 동영상', indent=False, layout=La…

Output()

Checkbox(value=True, description='[CSI4116] Intro to Computer Vision - Lecture 3 동영상', indent=False, layout=La…

...

Checkbox(value=True, description='[최신영상이해특강] Lecture 2 (3/9) 동영상', indent=False, layout=Layout(height='auto', …

Output()

Button(description='Download', style=ButtonStyle())

Output()

100%|██████████| 546/546 [01:04<00:00,  8.43it/s]
